# Sparkify AWS Deployment Notebook

Notebook to build optimised ML alogrithm to predict user churn in digital music services

## Setup

In [1]:
# imports
# pyspark sql
from pyspark.sql import SparkSession, Window, functions as F
from pyspark.sql.functions import from_unixtime, udf, col, when, isnan, desc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import min, max
from pyspark.sql.types import DoubleType

# pyspark ml
from pyspark.ml.feature import VectorAssembler, Normalizer, StringIndexer, OneHotEncoder
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
#from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1598622065746_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Starter code
from pyspark.sql import SparkSession

# Create spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# python imports
import datetime

from typing import NewType
pysparkdf = NewType('pysparkdf', object)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Read in full sparkify dataset
#event_data = "s3n://udacity-dsnd/sparkify/mini_event_data.json"
event_data = "s3n://udacity-dsnd/sparkify/mini_sparkify_event_data.json"
df = spark.read.json(event_data)
df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')

In [5]:
print("Started Spark version {}".format(spark.version))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Started Spark version 2.4.5-amzn-0

## Exploratory Data Analysis

In [6]:
print("Row 1:")
data_df = df
data_df.limit(2).head()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row 1:
Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')

In [7]:
# count users
print("Number of users:")
data_df.agg(countDistinct("userId")).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of users:
[Row(count(DISTINCT userId)=226)]

In [8]:
# count events
print("Number of log events:")
print((data_df.count(), len(data_df.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of log events:
(286500, 18)

In [9]:
print("start and end unix time:")
data_df.select(min("ts"), max("ts")).first()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

start and end unix time:
Row(min(ts)=1538352117000, max(ts)=1543799476000)

## Data Wrangling

In [10]:
def clean_df(data_df):
    """Remove non useful columns and drop missing values for user and session.
    """
    # lets remove some of the columns we don't think will be useful from data exploration
    cols_to_drop = ['firstName', 'lastName','artist', 'song', 'method', 'status', 'userAgent']
    user_log_df = data_df.drop(*cols_to_drop)
    
    # drop rows with missing info
    return user_log_df.dropna(how = "any", subset = ["userId", "sessionId"])

def unix_to_datetime(data_df):
    """ onvert unix timestamps to datetime.
    """
    # event unix to datetime
    data_df = data_df.withColumn("timestampDatetime",
                                         from_unixtime(user_log_valid.ts/1000,
                                                       format='yyyy-MM-dd HH:mm:ss'))
    # registration unix to datetime
    data_df = data_df.withColumn("registrationDatetime",
                                         from_unixtime(user_log_valid.registration/1000,
                                                       format='yyyy-MM-dd HH:mm:ss'))
    return data_df

def create_us_states(data_df):
    """Create US states column from location.
    """
    # we don't really want to drop these rows as the col isn't vital 
    # so replace missing values to allow split
    data_df = data_df.fillna({'location':'unknown'})

    # define UDFs
    # create state column
    loc_split = udf(lambda x: x.split(', ')[-1], StringType())

    # Sates seem to be appended, so take latest
    state_split = udf(lambda x: x.split('-')[-1], StringType())

    # apply UDFs
    data_df = data_df.withColumn("usStateAbbr",
                                 when(data_df.location.isNotNull(),
                                      loc_split(data_df.location)).otherwise(''))
    data_df = data_df.withColumn("usStateAbbr",
                                 when(data_df.usStateAbbr.isNotNull(),
                                      state_split(data_df.usStateAbbr)).otherwise(''))
    return data_df

def replace_missing_gender(data_df):
    """Replace missing gender with 'unknown'.
    """
    return data_df.fillna({'gender':'unknown'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
user_log_valid = clean_df(data_df)
user_log_valid = unix_to_datetime(user_log_valid)
user_log_valid = create_us_states(user_log_valid)
user_log_valid = replace_missing_gender(user_log_valid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
wrangled_data = user_log_valid.withColumnRenamed("auth", "authLevel").\
                               withColumnRenamed("length", "sessionLength_s").\
                               withColumnRenamed("level", "subLevel").\
                               withColumnRenamed("ts", "unixEventTS").\
                               withColumnRenamed("registration", "unixRegistrationTS")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Feature Engineering

In [13]:
def create_phase_feature(data_df: pysparkdf) -> pysparkdf:
    """Use the cancellation to identify churned users.
    """
    # flag any cancellation confirmation events in pages
    flag_cancellation_event = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())
    data_df = data_df.withColumn("churn", flag_cancellation_event("page"))

    # search for flags and fill user rows as churned
    windowval = Window.partitionBy("userId").orderBy(desc("unixEventTS")).rangeBetween(Window.unboundedPreceding, 0)
    return data_df.withColumn("label", Fsum("churn").over(windowval))

def avg_items_in_session(data_df: pysparkdf)-> pysparkdf:
    """Calculate avg items in session for each user.
    """
    # calculate metric and join dfs back together
    return data_df.join(data_df.groupBy('userId').avg('itemInSession'),
                               on='userId')

def avg_user_listening_time(data_df: pysparkdf) -> pysparkdf:
    """Calculate average listening time.
    """
    return data_df.join(data_df.groupBy('userId').avg('sessionLength_s'),
                               on='userId')

def recommendation_performance_good(data_df: pysparkdf) -> pysparkdf:
    """Number of positive recommendation events.
    """
    # flag events
    data_df = data_df.withColumn("recc_performance_good_events",
                                 when((data_df["page"] == 'Add to Playlist') |\
                                      (data_df["page"] == 'Add Friend') |\
                                      (data_df["page"] == 'Thumbs Up'),
                                       1).otherwise(0))
    # calculate number
    return data_df.join(data_df.groupBy('userId').sum('recc_performance_good_events'),
                               on='userId')

def recommendation_performance_bad(data_df: pysparkdf) -> pysparkdf:
    """Number of bad recommendation events.
    """
    # flag events
    data_df = data_df.withColumn("recc_performance_bad_events",
                                 when((data_df["page"] == 'Thumbs Down'),
                                       1).otherwise(0))
    # calculate number
    return data_df.join(data_df.groupBy('userId').sum('recc_performance_bad_events'),
                               on='userId')

def system_performance_bad(data_df: pysparkdf) -> pysparkdf:
    """Number of bad system events.
    """
    # flag events
    data_df = data_df.withColumn("sys_performance_bad",
                                 when((data_df["page"] == 'Help') |\
                                      (data_df["page"] == 'Upgrade') |\
                                      (data_df["page"] == 'Error'),
                                       1).otherwise(0))
    # calculate number                                
    return data_df.join(data_df.groupBy('userId').sum('sys_performance_bad'),
                               on='userId')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# create features
wrangled_data = create_phase_feature(wrangled_data)
wrangled_data = avg_items_in_session(wrangled_data)
wrangled_data = avg_user_listening_time(wrangled_data)
wrangled_data = recommendation_performance_good(wrangled_data)
wrangled_data = recommendation_performance_bad(wrangled_data)
wrangled_data = system_performance_bad(wrangled_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# create ml features df
ml_features_data = wrangled_data.select('label',
                                        'userId',
                                        'gender',
                                        'usStateAbbr',
                                        col('avg(itemInSession)').alias("avg_item_in_session"),
                                        col('avg(sessionLength_s)').alias('avg_session_length'),
                                        col('sum(recc_performance_good_events)').alias('num_good_recc'),
                                        col('sum(recc_performance_bad_events)').alias('num_bad_recc'),
                                        col('sum(sys_performance_bad)').alias('num_bad_sys'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
ml_features_data = ml_features_data.drop_duplicates()  # remove duplicates as features will be created on each event row
data_df.unpersist();  # clean up to free up space
wrangled_data.unpersist();  # clean up to free up space

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[userId: string, authLevel: string, gender: string, itemInSession: bigint, sessionLength_s: double, subLevel: string, location: string, page: string, unixRegistrationTS: bigint, sessionId: bigint, unixEventTS: bigint, timestampDatetime: string, registrationDatetime: string, usStateAbbr: string, churn: int, label: bigint, avg(itemInSession): double, avg(sessionLength_s): double, recc_performance_good_events: int, sum(recc_performance_good_events): bigint, recc_performance_bad_events: int, sum(recc_performance_bad_events): bigint, sys_performance_bad: int, sum(sys_performance_bad): bigint]

In [17]:
ml_features_data.limit(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[label: bigint, userId: string, gender: string, usStateAbbr: string, avg_item_in_session: double, avg_session_length: double, num_good_recc: bigint, num_bad_recc: bigint, num_bad_sys: bigint]

In [18]:
def onehot_encoder(ml_features_data):
    ## https://stackoverflow.com/questions/32277576/how-to-handle-categorical-features-with-spark-ml

    cols = ['gender', 'usStateAbbr']

    indexers = [
        StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c))
        for c in cols
    ]

    encoders = [
        OneHotEncoder(
            inputCol=indexer.getOutputCol(),
            outputCol="{0}_encoded".format(indexer.getOutputCol())) 
        for indexer in indexers
    ]

    assembler = VectorAssembler(
        inputCols=[encoder.getOutputCol() for encoder in encoders],
        outputCol="features2"
    )

    pipeline = Pipeline(stages=indexers + encoders + [assembler])
    return pipeline.fit(ml_features_data).transform(ml_features_data)

ml_features_data = onehot_encoder(ml_features_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
print('Shape of ML table:')
print((ml_features_data.count(), len(ml_features_data.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Shape of ML table:
(226, 14)

In [20]:
ml_features_data = ml_features_data.na.drop()  # catch any remaining nans

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
print('Shape of ML table:')
print((ml_features_data.count(), len(ml_features_data.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Shape of ML table:
(225, 14)

In [22]:
def vector_assembler(ml_features_data):

    # this vector is created in prep for ml
    assembler = VectorAssembler(inputCols=["avg_item_in_session",
                                           "avg_session_length",
                                           "num_good_recc",
                                           "num_bad_recc",
                                           "num_bad_sys",
                                           'gender_indexed',
                                           'usStateAbbr_indexed'],
                                outputCol="raw_features",)
                                #handleInvalid="skip")

    return assembler.transform(ml_features_data)

ml_features_data = vector_assembler(ml_features_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
def feature_scaler(ml_features_data):
    """apply scaler
    """
    scaler = Normalizer(inputCol="raw_features", outputCol="features")
    return scaler.transform(ml_features_data)

ml_df = feature_scaler(ml_features_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Machine Learning 

In [24]:
def train_set_evaluator(train):
    """Check for data skew
    """
    churned_count = train.filter(train['label']==1).count()
    non_churned_count = train.filter(train['label']==0).count()
    
    print("{} churned users".format(churned_count))
    print("{} non-churned users".format(non_churned_count))
    print("{} ratio of churned/non-churned users".format(churned_count/non_churned_count))

def custom_evaluation(results):
    """Customer function to evaluate result.
    """
    # Generic evaluation
    total_results = results.count()
    correct_pred = results.filter(results.label == results.prediction).count()
    incorrect_pred = results.filter(results.label != results.prediction).count()
    
    print('Total users predicted correctly: {}'.format(correct_pred))
    print('Total users predicted wrongly: {}'.format(incorrect_pred))
    print("Percentage predicted correct (%): {} \n".format((correct_pred/total_results)*100))
    
    # Correct churn predictions
    churn_correct = results.filter((results.label == 1) & (results.prediction == 1)).count()
    actual_churned_users = results.filter(results.label == 1).count()
    print('User churned and predicted to churn: {}'.format(churn_correct))
    print('Total users churned : {}'.format(actual_churned_users))
    print('Percent churned user events predicted correctly(%): {}\n'.format((churn_correct/actual_churned_users)*100))
    
    # Incorrect churn predictions
    churn_incorrect = results.filter((results.label == 0) & (results.prediction == 1)).count()
    print('User did not churn and predicted to: {}'.format(churn_incorrect))
    print('Percent churned users predicted incorrectly(%): {}\n'.format((churn_incorrect/total_results)*100))

def model_metrics(results):
    # ref: https://stackoverflow.com/questions/60772315/how-to-evaluate-a-classifier-with-apache-spark-2-4-5-and-pyspark-python

    # Create both evaluators
    evaluatorMulti = MulticlassClassificationEvaluator(labelCol="target",
                                                       predictionCol="prediction")
    evaluator = BinaryClassificationEvaluator(labelCol="target",
                                              rawPredictionCol="prediction",
                                              metricName='areaUnderROC')
    
    # Make predicitons
    results_cast = results.withColumn("target", col("label").cast(DoubleType()))
    predictionAndTarget = results_cast.select("target","prediction")

    # Get metrics
    acc = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "accuracy"})
    print("Accuracy: {}".format(acc))
    
    f1 = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "f1"})
    print("F1Score: {}".format(f1))
     
    weightedPrecision = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "weightedPrecision"})
    print("WeightedPrecision: {}".format(weightedPrecision))

    weightedRecall = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "weightedRecall"})
    print("WeightedRecall: {}".format(weightedRecall))
    
    auc = evaluator.evaluate(predictionAndTarget)
    print("AUC: {}".format(auc))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
ml_df = ml_df.select(['label', 'features'])
ml_df.limit(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[label: bigint, features: vector]

In [26]:
# train test split for ML validation
train, test =  ml_df.randomSplit([0.7, 0.3], seed=42)  # more equal fit to combat overfitting

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
train_set_evaluator(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

38 churned users
111 non-churned users
0.34234234234234234 ratio of churned/non-churned users

## Models

### Logistic Regression

In [38]:
print('Baseline Logistic Regression Model')
lr_model = LogisticRegression().fit(train)
lr_results = lr_model.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Baseline Logistic Regression Model

In [39]:
custom_evaluation(lr_results)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total users predicted correctly: 63
Total users predicted wrongly: 13
Percentage predicted correct (%): 82.89473684210526 

User churned and predicted to churn: 1
Total users churned : 14
Percent churned user events predicted correctly(%): 7.142857142857142

User did not churn and predicted to: 0
Percent churned users predicted incorrectly(%): 0.0

In [40]:
model_metrics(lr_results)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy: 0.7741935483870968
F1Score: 0.7117062284439034
WeightedPrecision: 0.7523236741388737
WeightedRecall: 0.7741935483870968
AUC: 0.5560283687943263

In [41]:
lr_results.unpersist();  # clean up to free up space!

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[label: bigint, features: vector, rawPrediction: vector, probability: vector, prediction: double]

### Random Forest Classifier

In [42]:
print('Baseline Random Forest Classifier Model')
rfc_model = RandomForestClassifier().fit(train)
rfc_results = rfc_model.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Baseline Random Forest Classifier Model

In [43]:
custom_evaluation(rfc_results)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total users predicted correctly: 65
Total users predicted wrongly: 11
Percentage predicted correct (%): 85.52631578947368 

User churned and predicted to churn: 3
Total users churned : 14
Percent churned user events predicted correctly(%): 21.428571428571427

User did not churn and predicted to: 0
Percent churned users predicted incorrectly(%): 0.0

In [44]:
model_metrics(rfc_results)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy: 0.7741935483870968
F1Score: 0.7117062284439034
WeightedPrecision: 0.7523236741388737
WeightedRecall: 0.7741935483870968
AUC: 0.5560283687943263

### Gradient Boosting

In [45]:
print('Baseline GBT Classifier Model')
gbt_model = GBTClassifier().fit(train)
gbt_results = gbt_model.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Baseline GBT Classifier Model

In [46]:
gbt_model.featureImportances

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparseVector(7, {0: 0.169, 1: 0.1259, 2: 0.1662, 3: 0.1693, 4: 0.1601, 5: 0.0569, 6: 0.1526})

In [47]:
custom_evaluation(gbt_results)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total users predicted correctly: 71
Total users predicted wrongly: 5
Percentage predicted correct (%): 93.42105263157895 

User churned and predicted to churn: 12
Total users churned : 14
Percent churned user events predicted correctly(%): 85.71428571428571

User did not churn and predicted to: 3
Percent churned users predicted incorrectly(%): 3.9473684210526314

In [48]:
model_metrics(gbt_results)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy: 0.6774193548387096
F1Score: 0.6892262281256589
WeightedPrecision: 0.7061633829510009
WeightedRecall: 0.6774193548387097
AUC: 0.6056737588652483

In [49]:
gbt_results.unpersist()  # clean up to free up space!

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[label: bigint, features: vector, rawPrediction: vector, probability: vector, prediction: double]

### Cross Validation

In [28]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# pipeline, just running it on classifier no transformations
gbt_cv_model = GBTClassifier()

pipeline = Pipeline(stages=[gbt_cv_model])

# set up param grid to iterate over
paramGrid = ParamGridBuilder()\
.addGrid(gbt_cv_model.maxDepth, [4, 7])\
.addGrid(gbt_cv_model.maxBins, [15, 40])\
.build()

# .addGrid(gbt_cv_model.stepSize, [0.05, 0.2])\
# default maxDepth: 5
# default maxBins: 32
# default stepSize: 0.1
# default maxIter: 20 .addGrid(gbt_cv_model.maxIter, [15, 30, 40])\

# cv_evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',
#                                                     labelCol='label',
#                                                     metricName='f1')

cv_evaluator = MulticlassClassificationEvaluator(predictionCol='prediction',
                                                 labelCol='label',
                                                 metricName='f1')

# set up crossvalidator to tune parameters and optimize, returns best model
crossval = CrossValidator(estimator=pipeline,
                         estimatorParamMaps=paramGrid,
                         evaluator=cv_evaluator,
                         numFolds=3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
cvModel = crossval.fit(train)  # train model
cv_results = cvModel.transform(test)  # apply model on test data

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
custom_evaluation(cv_results)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total users predicted correctly: 67
Total users predicted wrongly: 9
Percentage predicted correct (%): 88.1578947368421 

User churned and predicted to churn: 8
Total users churned : 14
Percent churned user events predicted correctly(%): 57.14285714285714

User did not churn and predicted to: 3
Percent churned users predicted incorrectly(%): 3.9473684210526314

In [35]:
model_metrics(cv_results)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy: 0.6935483870967742
F1Score: 0.6812622399586151
WeightedPrecision: 0.6719354838709678
WeightedRecall: 0.6935483870967742
AUC: 0.5482269503546099

In [36]:
evaluator=MulticlassClassificationEvaluator(metricName="f1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
evaluator.evaluate(cv_results)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.6812622399586151